In [1]:
from assignment1 import string_distance as rust_string_distance
from string import ascii_lowercase
from random import choice, randint
from time import time 
import ctypes

In [2]:
def python_string_distance(a, b):
    d = [[0 for _ in range(len(b) + 1)] for _ in range(len(a) + 1)]
    for i in range(1, len(a) + 1): 
        for j in range(1, len(b) + 1): 
            cost = 1 if a[i - 1] != b[j - 1] else 0
            d[i][j] = min(d[i][j-1] + 1, d[i-1][j] + 1, d[i-1][j-1] + cost)

            if i > 1 and j > 1 and a[i-1] == b[j-2] and a[i-2] == b[j-1]: 
                d[i][j] = min(d[i][j], d[i-2][j-2] + 1)
                
    # print('\n'.join(map(str, d)))
    return d[len(a)][len(b)]

class CWrapper:
    def __init__(self):
        self.lib = ctypes.CDLL("./libexample.so")

        self.lib.distance.argtypes = [ctypes.c_char_p, ctypes.c_char_p]
        self.lib.distance.restype = ctypes.c_int

    def __call__(self, a, b):
        a_bytes = a.encode('utf-8')
        b_bytes = b.encode('utf-8')

        res = self.lib.distance(a_bytes, b_bytes)
        return res
    
c_string_distance = CWrapper()  

In [3]:
dataset = []
for _ in range(1000): 
    len_a = randint(2, 1000)
    len_b = randint(2, 1000)
    
    a = ''.join([choice(ascii_lowercase) for _ in range(len_a)])
    b = ''.join([choice(ascii_lowercase) for _ in range(len_b)])
    dataset.append((a, b))
# dataset[:3]

In [4]:
# for a, b in dataset:
#     assert python_string_distance(a, b) == rust_string_distance(a, b) == c_string_distance(a, b)
    
python_time = 0
# start = time()
# for a, b in dataset: 
#     python_string_distance(a, b)    
# python_time = time() - start
# print(f'Python: {python_time:.3f} seconds')

start = time()
for a, b in dataset: 
    rust_string_distance(a, b)    
rust_time = time() - start
print(f'Rust:      {rust_time:.3f} seconds')

start = time()
for a, b in dataset: 
    c_string_distance(a, b)    
c_time = time() - start
print(f'C:           {c_time:.3f} seconds')

print()
print(f'Improvement of Rust: {python_time / rust_time:.2f} times')
print(f'Improvement of C:       {python_time / c_time:.2f} times')

print()
print(f'C/Rust = {c_time/rust_time:.3f}')

Rust:      0.755 seconds
C:           0.407 seconds

Improvement of Rust: 0.00 times
Improvement of C:       0.00 times

C/Rust = 0.538
